In [1]:
import sys
sys.path.append('../')

In [2]:
import os

import numpy as np
import pandas as pd
import pickle as pkl

import matplotlib.pyplot as plt

In [3]:
import blo.params as blo_params
from blo.utils import load_problem, factory_get_path

In [4]:
def get_results(param_vals, data_dir, model_type, vf_constr_type, approx_type):
    """ Get results for ml algorithm. """
    result_dir = 'results'
    result_dir = f'{data_dir}{result_dir}/'

    result_files = os.listdir(result_dir)
    result_files = list(filter(lambda x: f"m-{model_type}" in x, result_files))
    result_files = list(filter(lambda x: f"v-{vf_constr_type}" in x, result_files))
    result_files = list(filter(lambda x: f"a-{approx_type}" in x, result_files))

    print("Number of files:", len(result_files))

    results = {}

    for i in param_vals['inst_ids']:
        inst_str = f"_i-{i}_"
        f = list(filter(lambda x: inst_str in x, result_files))

        f = f[0]
        f = result_dir + f
    
        with open(f, 'rb') as p:
            results[i] = pkl.load(p)
        
    return results

In [5]:
model_type = "inst_encoder"

param_vals = {
    'inst_ids' : list(range(1,11)),
    'n' : [30],
}

In [7]:
problem = 'dr'
data_dir = f'../data/{problem}/'

In [8]:
nn_lower_results = get_results(param_vals, data_dir, model_type, "slack", "lower")

Number of files: 10


In [9]:
nn_upper_results = get_results(param_vals, data_dir, model_type, "slack", "upper")

Number of files: 10


In [10]:
# get baseline results
solver_results = {}
for k, v in nn_lower_results.items():
    solver_results[k] = nn_lower_results[k]['baseline_results']

In [11]:
# columns for table
cols = ['nn_l_obj', 'nn_u_obj',  's_obj',
        'nn_l_mre',  'nn_u_mre',  's_mre',
        'nn_l_time', 'nn_u_time',  's_time', 
       ]

In [12]:
# indicies for table
indicies = list(range(1,11)) 
indicies.append("Averaged")

rel_errors = {}
rel_errors["nn_u"] = []
rel_errors["nn_l"] = []
rel_errors["s"] = []

In [13]:
df = pd.DataFrame(index=indicies, columns=cols)

In [14]:
for i in param_vals['inst_ids']:
    
    # results for solver results
    solver_time = solver_results[i]['runtime']
    solver_obj = solver_results[i]['obj']

    # slack results
    nn_lower_time = nn_lower_results[i]['time']
    nn_lower_obj = nn_lower_results[i]['leader_obj']

    # upper results
    nn_upper_time = nn_upper_results[i]['time']
    nn_upper_obj = nn_upper_results[i]['leader_obj']

    # comptue relative error for each method
    best_obj = np.max([nn_upper_obj, nn_lower_obj, solver_obj])
    
    solver_re = 100 * (best_obj - solver_obj) / best_obj
    nn_lower_re = 100 * (best_obj - nn_lower_obj) / best_obj
    nn_upper_re = 100 * (best_obj - nn_upper_obj) / best_obj

    rel_errors["nn_u"].append(nn_upper_re)
    rel_errors["nn_l"].append(nn_lower_re)
    rel_errors["s"].append(solver_re)
    
    df['nn_l_obj'][i] = nn_lower_obj
    df['nn_u_obj'][i] = nn_upper_obj
    df['s_obj'][i] = float(solver_obj)

    df['nn_l_mre'][i] = nn_lower_re
    df['nn_u_mre'][i] = nn_upper_re
    df['s_mre'][i] = solver_re

    df['nn_l_time'][i] = nn_lower_time
    df['nn_u_time'][i] = nn_upper_time
    df['s_time'][i] = solver_time

In [15]:
df['nn_l_obj']["Averaged"] = np.mean(df['nn_l_obj'][indicies[:-1]])
df['nn_u_obj']["Averaged"] = np.mean(df['nn_u_obj'][indicies[:-1]])
df['s_obj']["Averaged"] = np.mean(df['s_obj'][indicies[:-1]])

df['nn_l_mre']["Averaged"] = np.mean(rel_errors["nn_l"])
df['nn_u_mre']["Averaged"] = np.mean(rel_errors["nn_u"])
df['s_mre']["Averaged"] = np.mean(rel_errors["s"])

df['nn_l_time']["Averaged"] = np.mean(df['nn_l_time'][indicies[:-1]])
df['nn_u_time']["Averaged"] = np.mean(df['nn_u_time'][indicies[:-1]])
df['s_time']["Averaged"] = np.mean(df['s_time'][indicies[:-1]])

In [16]:
df

,nn_l_obj,nn_u_obj,s_obj,nn_l_mre,nn_u_mre,s_mre,nn_l_time,nn_u_time,s_time
1,34356.0,59524.0,47206.0,42.282105,0.0,20.694174,0.091573,1.438129,3600.087101
2,33713.0,54764.0,39526.0,38.439486,0.0,27.824848,0.117806,1.518419,3600.082969
3,36717.0,66967.0,46792.0,45.171502,0.0,30.126779,0.138996,2.850251,3600.07455
4,36414.0,54908.0,44486.0,33.681795,0.0,18.980841,0.068614,1.683495,3637.234302
5,33090.0,59627.0,43355.0,44.505006,0.0,27.289651,0.099623,1.961304,3600.070555
6,36691.0,56603.0,39006.0,35.178347,0.0,31.088458,0.081683,2.934948,3600.09992
7,31354.0,55569.0,43443.0,43.576454,0.0,21.821519,0.091259,1.575713,3600.142473
8,35710.0,54414.0,39839.0,34.373507,0.0,26.785386,0.093465,0.866674,3600.102146
9,38961.0,61869.0,45288.0,37.026621,0.0,26.800175,0.162728,4.549298,3600.163771
10,36965.0,60488.0,43194.0,38.888705,0.0,28.590795,0.120635,3.572193,3600.099877


In [17]:
def latex_results(df_):
    """ Generate Latex table for results.  """

    df = df_.copy()

    df = df.rename(columns={
        'nn_l_obj' : '\\mls',
        'nn_u_obj' : '\\mlu',
        's_obj' : '\\drbaseline',
 
        'nn_l_mre' : '\\mls\ ',
        'nn_u_mre' : '\\mlu\ ',
        's_mre' : '\\drbaseline\ ',

        'nn_l_time' : '\\mls\ \ ',
        'nn_u_time' : '\\mlu\ \ ',
        's_time' : '\\drbaseline\ \ ',

    })

    df = df.style.format(thousands=',', precision=2)        
    
    # bold minimum value in table
    df = df.highlight_max(subset=['\\mls',     '\\mlu',     '\\drbaseline'], props='textbf:--rwrap;', axis=1)
    df = df.highlight_min(subset=['\\mls\ ',   '\\mlu\ ',   '\\drbaseline\ '], props='textbf:--rwrap;', axis=1)
    df = df.highlight_min(subset=['\\mls\ \ ', '\\mlu\ \ ', '\\drbaseline\ \ '], props='textbf:--rwrap;', axis=1)

    # caption
    end_str = '}\n\\caption{Donor Receipient: Objectives, gaps, and times for all instances. '
    end_str += 'Each row corresponds to a single instance from \\jd{ADD citation and description}.  ' 
    end_str += '\\mls{} is the ML formulations with slack for the lower-level approximation.  ' 
    end_str += '\\mlu{} is the ML formulation for the upper-level approximation. }\n'
    end_str += '\\label{tab:dr_results}\n'
    end_str += '\\end{table*}' 


    latex_str = df.to_latex(column_format='l|rrr|rrr|rrr').replace('_','\_').replace('nan', '-').replace('#', '\#')

    #row_id = "($||,\\gamma,"
    # row_id += "\\eta,"
    #row_id += "\\epsilon,\\delta,\\eta,d\_{ratio},a\_{ratio}$)"
    row_id = "Instance \#"
    
    latex_str2 = (latex_str.splitlines())
    latex_str2.insert(1, '\\toprule')
    latex_str2.insert(2, row_id + ' & \multicolumn{3}{c|}{Objective}  &  \multicolumn{3}{c|}{Relative Error (\%)}  & \multicolumn{3}{c}{Times} \\\\')
    latex_str2.insert(4, '\\midrule')
    latex_str2.insert(-1, '\\bottomrule')
    latex_str2.insert(0, '\\begin{table*}[h]\\centering\\resizebox{0.5\\textwidth}{!}{')
    latex_str2[-1] += end_str

    print("\n".join(latex_str2))
    print()

In [18]:
latex_results(df)

\begin{table*}[h]\centering\resizebox{0.5\textwidth}{!}{
\begin{tabular}{l|rrr|rrr|rrr}
\toprule
Instance \# & \multicolumn{3}{c|}{Objective}  &  \multicolumn{3}{c|}{Relative Error (\%)}  & \multicolumn{3}{c}{Times} \\
 & \mls & \mlu & \drbaseline & \mls\  & \mlu\  & \drbaseline\  & \mls\ \  & \mlu\ \  & \drbaseline\ \  \\
\midrule
1 & 34,356.00 & \textbf{59,524.00} & 47,206.00 & 42.28 & \textbf{0.00} & 20.69 & \textbf{0.09} & 1.44 & 3,600.09 \\
2 & 33,713.00 & \textbf{54,764.00} & 39,526.00 & 38.44 & \textbf{0.00} & 27.82 & \textbf{0.12} & 1.52 & 3,600.08 \\
3 & 36,717.00 & \textbf{66,967.00} & 46,792.00 & 45.17 & \textbf{0.00} & 30.13 & \textbf{0.14} & 2.85 & 3,600.07 \\
4 & 36,414.00 & \textbf{54,908.00} & 44,486.00 & 33.68 & \textbf{0.00} & 18.98 & \textbf{0.07} & 1.68 & 3,637.23 \\
5 & 33,090.00 & \textbf{59,627.00} & 43,355.00 & 44.51 & \textbf{0.00} & 27.29 & \textbf{0.10} & 1.96 & 3,600.07 \\
6 & 36,691.00 & \textbf{56,603.00} & 39,006.00 & 35.18 & \textbf{0.00} & 31.09 & \text

# Data Collection + Training Time

In [19]:
class A():
    pass

In [20]:
args = A()
args.v = v
model_type = "inst_encoder"

In [21]:
param_vals

{'inst_ids': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'n': [30]}

In [22]:
ml_times = {}
u_tr_times = {}
l_tr_times = {}

for n in param_vals['n']:

    args.problem = f"dr_{n}_hard"
    get_path = factory_get_path(args)
    cfg = getattr(blo_params, args.problem)
    cfg.data_path = f'../data/'

    # load dataset
    fp_data = get_path(cfg.data_path, cfg, "ml_data")
    with open(fp_data, 'rb') as pf:
        ml_dataset = pkl.load(pf)

    ml_times[n] = ml_dataset['total_time']
    
    
    for approx_type in ["lower", "upper"]:
        fp_nn_res = get_path(cfg.data_path, cfg, f"nn_res_{model_type}_{approx_type}", suffix="pkl")
        with open(fp_nn_res, 'rb') as pf:
            nn_res = pkl.load(pf)

        if approx_type == "lower":
            l_tr_times[n] = nn_res['train_time']
        else:
            u_tr_times[n] = nn_res['train_time']

In [23]:
l_tr_times

{30: 1768.8261532783508}

In [24]:
u_tr_times

{30: 1784.1526777744293}

In [25]:
ml_times

{30: 1939.2453722953796}